# Let's Begin
Let's import some modules that will be used.

In [1]:
# for API work
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time 
import pprint as pp
import numpy as np

# for DB work
from sqlalchemy import create_engine
from pandas.io import sql

## SQL Credentials

In [2]:
# inserting data into database
# this will overwrite the database though,
# so if you want to merge data, a different operation
# or table might be the better play
# if exists also has "append"

hostname=''
dbname=''
uname=''
pwd=''

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=uname, pw=pwd))

## First let's see the current stage of data we have from Kaggle
The data shown below is provided by Kaggle, but we noticed that some song's had invalid names and characters, and we also though it'd be better to get new data while using song ID provided in kaggle's dataset.

In [3]:
# the original data
df=pd.read_csv('./data/data.csv')
df

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
2,0.993000,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
3,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
4,0.295000,['Mixe'],0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174384,0.009170,"['DJ Combo', 'Sander-7', 'Tony T']",0.792,147615,0.866,0,46LhBf6TvYjZU2SMvGZAbn,0.000060,6,0.1780,-5.089,0,The One,0,2020-12-25,0.0356,125.972,0.1860,2020
174385,0.795000,['Alessia Cara'],0.429,144720,0.211,0,7tue2Wemjd0FZzRtDrQFZd,0.000000,4,0.1960,-11.665,1,A Little More,0,2021-01-22,0.0360,94.710,0.2280,2021
174386,0.806000,['Roger Fly'],0.671,218147,0.589,0,48Qj61hOdYmUCFJbpQ29Ob,0.920000,4,0.1130,-12.393,0,Together,0,2020-12-09,0.0282,108.058,0.7140,2020
174387,0.920000,['Taylor Swift'],0.462,244000,0.240,1,1gcyHQpBQ1lfXGdhZmWrHP,0.000000,0,0.1130,-12.077,1,champagne problems,69,2021-01-07,0.0377,171.319,0.3200,2021


### Observation
We have ```174389 rows```, but we don't know how recent this data is. So we'll get new data using Spotify API and Spotipy module as well. We can do this by using only ID's from the existing data and running it through Spotify's API to get updated result.

## Spotify credentials

In [4]:
#api keys stuff 
client_id = ''
client_secret = ''

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Get track ids
Function below is used to get song ids from a given playlist IDS. We already have 174389 ids. But we dont't know how many of them are duplicates and if they're still valid. So, we are going to get new track ids form public playlists of Spotify. The playlists are all randomly searched manually, we then copy the playlists link and select all the uncessary part of a link and remove them, which will leave us with playlists ids only. 
    
Example :
    
    Link 1 - https://open.spotify.com/playlist/2bRMdtifHI8VvkW5Sn28s4?si=4bc7b896f72c4d3d
    Playlist ID 1 - 2bRMdtifHI8VvkW5Sn28s4
    
    Link 2 - spotify:playlist:1lKSqQGrGLPNCASmKQS1qj
    Playlist ID 2 - 1lKSqQGrGLPNCASmKQS1qj
    
    
Then just run those playlist ids through the function provided below, which will give us individual track ids.

### First way to get song ids was through Kaggle Dataset

In [5]:
#ids from the kaggle dataset
get_ids=list(df['id']) 
print("Song ids in Kaggle dataset alone is ", len(get_ids))
(get_ids[:5])

Song ids in Kaggle dataset alone is  174389


['0cS0A1fUEUd1EW3FcF8AEI',
 '0hbkKFIJm7Z05H8Zl9w30f',
 '11m7laMUgmOKqI3oYzuhne',
 '19Lc5SfJJ5O1oaxY0fpwfh',
 '2hJjbsLCytGsnAHfdsLejp']

### Getting more Track ids
We got track ids from kaggle dataset, but we also have to get track ids from the playlists we collected

In [6]:
#use this to get song ids from a desired Playlist, you'll need a playlist ID. 
def show_tracks (results,uriArray):
    for i, item in enumerate(results['items']):
        track = item['track']
        uriArray.append(track['id'])
        
# getTrackIDs takes in userid , playlist_id. 
# User ID is useful when u want to get playlist and songids of a specific user
def getTrackIDs(user, playlist_id):
    trackId= []
    playlist = sp.user_playlist(user, playlist_id)
    tracks = playlist['tracks']
    show_tracks(tracks,trackId)
    while tracks['next']:
        tracks = sp.next(tracks)
        show_tracks(tracks,trackId)
    return trackId



######
#song ids r given below
trackids=getTrackIDs('','2bRMdtifHI8VvkW5Sn28s4')
trackids

['3aFUQpntBHJJRzARB85rOp',
 '3hTE8wuBhQ7bTkPeCEOWWA',
 '0BBdvQIUYLFFhws3RRUvbw',
 '1FdfQx7PXvbUOcZOlQYdYF',
 '3pVVFQ4D1yRBpztapn2hHG',
 '2B1jWeJLF5yWH84nQt4PQ2',
 '6Xg3QvvrCaabYqFsrH2Lrn',
 '7pjzjFw0Tq6syGPlRQ4v1q',
 '2kQJtU6D35IRPsm6Cu1flA',
 '7rG7jgVJF0wbd5lI4GnJrI',
 '2aG7VntODEsrI2RDz3hdbG',
 '5sGcImZ0nh5dRHmwfcY8VN',
 '6LOOqW1iQwYNWF0Gtilu7H',
 '0isPvCWiORiZVCJPUKUA9A',
 '134tymGB5lCH4onVDqeKaf',
 '36HoI81LLrNCxfXo8P1gry',
 '62yjJcONaywby93lw8AgQv',
 '0SqIPA2h44hSZy89rTnaaN',
 '4voQwP4mvZfEPqOFbdeZmV',
 '2EyPWfDjkr5tb7TJGT0ins',
 '5M2urKCPnhOPNeGCi6Jm9W',
 '54NLjntUle92cVAQvzBcXs',
 '0JAwAgwSP2GT9dfngdvUFZ',
 '2Lbt7W0iFntHzRqbPOmAT5',
 '3uGYjUZ5TMaYZV81mUqhgP',
 '2SWXY464AgSEJubk9WHaKp',
 '4r69QKaIPJ1dp0aKdYu5UM',
 '1aLropRwLRuHPeaYOvsIvB',
 '4IawNNTvYujmjZMUDuTrdU',
 '17GiXm5512DkSVOeXwEt77',
 '0eAZ56mzI2ioVYGCu5weWd',
 '7IdZyHbuEXKIs6QhdvHKvi',
 '6cHDbFRk323AWqdAkLot8F',
 '6B4NOTnzluko6NkWZYgHks',
 '0aJYsiLUOkiZbmXhaBgwNA',
 '7nDJRlWjzrQScIU1ZhVfU8',
 '0vafwkDvYxdW5NkSVp0t3M',
 

In [13]:
# These are few playlist Ids
playlists_IDS=["3siczhp0oLir1feGHC9gbq",
"5hykBrJye8KjaGv038m5pm",
"583Cg3GvvCxNSm9sxYreVw",
"36GYj7s5zAnLT1Ec6TFOUA",
"21J20PgNUj49fSp2UOfawo"]

In [15]:
#song ids r given below
track_ids=[]
for i in playlists_IDS:
    try:
        track_ids.append(getTrackIDs('1248528118',i))
    except:
        print(i)
len(track_ids)

5

In [16]:
# all the track ids are in a list of list
print(len(track_ids))
print(len(track_ids[0]))

5
42


In [17]:
#Unpacking list of lists
flat_list = [item for sublist in track_ids for item in sublist]
len(flat_list)

10929

In [18]:
len(flat_list)

10929

In [10]:
# all the ids optained from playlists and not from kaggle are stored in songids table
df = pd.DataFrame(flat_list, columns = ['ids'])

#'exp' table just for testing purposes
df.to_sql('exp', engine, if_exists='replace', index=False)

### Getting Track Features
The functions below is getting us song's features like energy, loudness, tempo and etc.

In [19]:
#for song features
def getTrackFeatures(id):
    meta = sp.track(id)
    features = sp.audio_features(id)

    #meta
    ids = meta['id']
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms'] #return of this will be divided by 60000 to convert from ms to minutes
    popularity = meta['popularity']

    # features
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']
    key = features[0]['key']
    valence = features[0]['valence']
    


    track = [ids, name, album, artist, release_date, length/60000, popularity, danceability, acousticness, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature,key,valence]
    return track

#### Example
Below is example output of a track id ```3aFUQpntBHJJRzARB85rOp```.

Outputs shown is a songs feautre. The respective order is - ids, name, album, artist, release_date, length, popularity, danceability, acousticness, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature,keys]

In [20]:
getTrackFeatures('3aFUQpntBHJJRzARB85rOp')

['3aFUQpntBHJJRzARB85rOp',
 'Food For Woofers - Ivan Carsten Rmx',
 'Hardstyle Revolution Vol. 2',
 'Various Artists',
 '2009-06-08',
 5.647766666666667,
 0,
 0.543,
 0.00101,
 0.845,
 0.874,
 0.615,
 -4.821,
 0.0494,
 150.009,
 4,
 9,
 0.221]

# Dividing the data gathering Task

In the end, we ended up with more than **300,000** song ids.

Since the list of track is very lengthy, we know it's going to be time consuming to get new updated data from Spotify. So we decided to divide track ids between two people, with each gettin approximately **150,000** song ids.

# Gathering ingredients
Now for the gathering part
Took approximately 48 hours, to collect data.

In [ ]:
# loop over track ids 
tracks = []
invalids=[]

#To see how long does collecting information for 85,000 songs take
start_time = time.time()


# following loop was ran 4 different time with limit being 
# get_ids[89998:]; get_ids[:89998]; get_ids[100000:]; get_ids[:100000]
for i in get_ids[89998:]:
    try:
        track = getTrackFeatures(i)
        tracks.append(track)
    except:
        invalids.append(i)

### create dataset

# if ids are valids, then the information is colleced in df and 
# sent to 'users' table of SQL
# the output is stored in 4 different SQL tables
# test, users, table_A, table_B

df = pd.DataFrame(tracks, columns = ['ids', 'name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature','keys','valence'])
df.to_sql('users', engine, if_exists='replace', index=False)

# if ids are invalids, then the information is colleced in df1 and 
# sent to 'invalids' table of SQL
df1 = pd.DataFrame(invalids, columns = ['ids'])
df1.to_sql('invalids', engine, if_exists='append', index=False)

print("--- %s seconds ---" % (time.time() - start_time))


In [16]:
df_b = pd.read_sql('SELECT * FROM users', con=engine)
df_b1 = pd.read_sql('SELECT * FROM test', con=engine)
df_b2 = pd.read_sql('SELECT * FROM table_A', con=engine)
df_b3 = pd.read_sql('SELECT * FROM table_B', con=engine)

In [19]:
df_b.head()

,ids,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,3Y4rNagSf2IukhcOTWYJuV,Death Blooms,L.D. 50,Mudvayne,2000-08-13,291933,49,0.275,0.000355,0.974,0.002400,0.1580,-4.624,0.1920,174.095,4
1,5Gp0qerHdLx33Q3csywzJh,Dying Swan Year 2000,Complete Discography,Ramases,2014-04-22,45546,1,0.734,0.853000,0.108,0.000000,0.0741,-5.586,0.0867,108.027,1
2,3qd4eFbXvTs0xsgDnExC0I,I Was Country When Country Wasn't Cool - (Duet...,20th Century Masters: The Millennium Collectio...,Barbara Mandrell,2000-01-01,220373,45,0.607,0.078900,0.555,0.000179,0.9290,-11.347,0.0314,104.937,4
3,4qnab6Ta94xTME6f3tPVfi,2000 Years Ago - original,2000 Years Ago EP,Pan Papason,2011-07-18,467439,0,0.684,0.000975,0.998,0.441000,0.0607,-5.740,0.0445,141.977,4
4,5PIgwTDsrWXvW7VU6QCEOf,Ain't No Mountain High Enough - Edit Version,20th Century Masters: The Millennium Collectio...,Diana Ross,2000-01-01,217493,46,0.401,0.384000,0.783,0.003190,0.4680,-5.929,0.0465,99.274,4


In [20]:
df_b1.head()

,ids,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,0cS0A1fUEUd1EW3FcF8AEI,Keep A Song In Your Soul,Crazy Blues: The Best Of Mamie Smith,Mamie Smith,1920,168333,11,0.598,0.991000,0.224,0.000522,0.3790,-12.628,0.0936,149.976,4
1,0hbkKFIJm7Z05H8Zl9w30f,I Put A Spell On You,I Put A Spell On You (Remastered),Screamin' Jay Hawkins,1920-01-05,150200,6,0.852,0.643000,0.517,0.026400,0.0809,-7.261,0.0534,86.889,4
2,11m7laMUgmOKqI3oYzuhne,Golfing Papa,Crazy Blues: The Best Of Mamie Smith,Mamie Smith,1920,163826,4,0.647,0.993000,0.186,0.000018,0.5190,-12.098,0.1740,97.600,4
3,19Lc5SfJJ5O1oaxY0fpwfh,True House Music - Xavier Santos & Carlos Gomi...,This Is True House Music,Oscar Velazquez,1920-01-01,422086,17,0.730,0.000173,0.798,0.801000,0.1280,-7.311,0.0425,127.997,4
4,2hJjbsLCytGsnAHfdsLejp,Xuniverxe,Xuniverxe,Mixe,1920-10-01,165224,2,0.704,0.295000,0.707,0.000246,0.4020,-6.036,0.0768,122.076,4


In [21]:
df_b2.head()

,ids,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,49FYlytm3dAAraYgpoJZux,Umbrella,Good Girl Gone Bad: Reloaded,Rihanna,2008-06-02,275986,82,0.583,0.008640,0.829,0.000000,0.0426,-4.603,0.1340,174.028,4
1,5BHtT1xpC44pGy1J7RKY4T,The Sweet Escape,The Sweet Escape,Gwen Stefani,2006-12-01,246466,63,0.756,0.191000,0.770,0.000000,0.1780,-3.502,0.0343,119.961,4
2,2Lhdl74nwwVGOE2Gv35QuK,Cupid's Chokehold / Breakfast in America,The Papercut Chronicles,Gym Class Heroes,2005-02-22,243773,70,0.720,0.237000,0.744,0.000000,0.2550,-6.938,0.1200,78.953,4
3,5JkTN6THtYH6bpOoHjOysc,Foundations,Made of Bricks,Kate Nash,2007-08-06,245173,60,0.502,0.088600,0.904,0.000000,0.0839,-4.010,0.0774,84.461,4
4,45hOioMDJktr86iKDHC8gr,Girlfriend,The Best Damn Thing (Expanded Edition),Avril Lavigne,2007-04-17,216131,75,0.560,0.000738,0.959,0.000206,0.2090,-2.433,0.1020,163.983,4


In [22]:
df_b3.head()

,ids,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,0jiW3PNiHJxOhWh9oPBJ7m,Held Me Down,Perfect Timing,NAV,2017-07-21,228106,62,0.826,0.4830,0.422,0.000000,0.1080,-8.217,0.1100,119.997,4.0
1,2YyLNljkMy2viVcXoFGTDY,Tears of Gold,Tears of Gold,Faouzia,2019-11-20,176117,68,0.587,0.1280,0.664,0.000000,0.3230,-5.698,0.0523,149.957,4.0
2,6i5jBmnB22alUpJ7hpznuj,Born Without a Heart,Born Without a Heart,Faouzia,2019-03-28,198896,64,0.699,0.0307,0.613,0.000000,0.1750,-5.827,0.0328,122.934,4.0
3,4wk9RMAmnUqFdgpBjplA4g,You Don't Even Know Me,You Don't Even Know Me,Faouzia,2019-09-18,179396,57,0.577,0.2440,0.663,0.000077,0.0795,-4.788,0.0374,165.938,4.0
4,1ds2QsfhAAfRiaFMGDzrdb,What Do You Mean?,What Do You Mean?,Justin Bieber,2015-08-28,207546,0,0.842,0.6170,0.564,0.000240,0.0912,-8.042,0.1050,124.984,4.0


In [54]:
print('Users table length \t',len(df_b))
print('Test table length \t',len(df_b1))
print('Table_A table length \t',len(df_b2))
print('Table_B table length \t',len(df_b3))
print('Total we have now: \t',len(df_b)+len(df_b1)+len(df_b2)+len(df_b3))

Users table length 	 84391
Test table length 	 89998
Table_A table length 	 99721
Table_B table length 	 101341
Total we have now: 	 375451


# Combining database tables

We'll just use the concat method, but the index will repeat from both ```df_b```, ```df_b1```, ```df_b2```, and```df_b3```. So, we will have to reset the index counts.

In [28]:
frames=[df_b,df_b1,df_b2,df_b3]
fdb=pd.concat(frames)
fdb.reset_index(drop=True)

,ids,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,3Y4rNagSf2IukhcOTWYJuV,Death Blooms,L.D. 50,Mudvayne,2000-08-13,291933,49,0.275,0.000355,0.974,0.002400,0.1580,-4.624,0.1920,174.095,4.0
1,5Gp0qerHdLx33Q3csywzJh,Dying Swan Year 2000,Complete Discography,Ramases,2014-04-22,45546,1,0.734,0.853000,0.108,0.000000,0.0741,-5.586,0.0867,108.027,1.0
2,3qd4eFbXvTs0xsgDnExC0I,I Was Country When Country Wasn't Cool - (Duet...,20th Century Masters: The Millennium Collectio...,Barbara Mandrell,2000-01-01,220373,45,0.607,0.078900,0.555,0.000179,0.9290,-11.347,0.0314,104.937,4.0
3,4qnab6Ta94xTME6f3tPVfi,2000 Years Ago - original,2000 Years Ago EP,Pan Papason,2011-07-18,467439,0,0.684,0.000975,0.998,0.441000,0.0607,-5.740,0.0445,141.977,4.0
4,5PIgwTDsrWXvW7VU6QCEOf,Ain't No Mountain High Enough - Edit Version,20th Century Masters: The Millennium Collectio...,Diana Ross,2000-01-01,217493,46,0.401,0.384000,0.783,0.003190,0.4680,-5.929,0.0465,99.274,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375446,6tohHT5sQRMjdWHMNn190u,,,,2014-03-28,312091,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
375447,59o6ojGNGJOYiVJSzC6Lsa,Memories (feat. Kid Cudi) - 2021 Remix,Memories (feat. Kid Cudi) [2021 Remix],David Guetta,2021-01-08,162992,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
375448,57lJqIRbNqqucBNtnVsIuf,While I'm Here (feat. Polo G),A Demon In 6lue,Lil Loaded,2020-10-08,209424,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
375449,3tc9Kbro2cpggcOWTRTS5x,Bogota - 2020 Refix,Immortality EP,Angerfist,2020-05-08,152228,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Little Cleaning
Remove Duplicate ids and NaN rows

In [34]:
#drop any rows where we might have NaN values in any cell
fdb = fdb.dropna()

#remove duplicate IDs 
fdb = fdb.drop_duplicates(subset=['ids'])
fdb

,ids,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,3Y4rNagSf2IukhcOTWYJuV,Death Blooms,L.D. 50,Mudvayne,2000-08-13,291933,49,0.275,0.000355,0.974,0.002400,0.1580,-4.624,0.1920,174.095,4.0
1,5Gp0qerHdLx33Q3csywzJh,Dying Swan Year 2000,Complete Discography,Ramases,2014-04-22,45546,1,0.734,0.853000,0.108,0.000000,0.0741,-5.586,0.0867,108.027,1.0
2,3qd4eFbXvTs0xsgDnExC0I,I Was Country When Country Wasn't Cool - (Duet...,20th Century Masters: The Millennium Collectio...,Barbara Mandrell,2000-01-01,220373,45,0.607,0.078900,0.555,0.000179,0.9290,-11.347,0.0314,104.937,4.0
3,4qnab6Ta94xTME6f3tPVfi,2000 Years Ago - original,2000 Years Ago EP,Pan Papason,2011-07-18,467439,0,0.684,0.000975,0.998,0.441000,0.0607,-5.740,0.0445,141.977,4.0
4,5PIgwTDsrWXvW7VU6QCEOf,Ain't No Mountain High Enough - Edit Version,20th Century Masters: The Millennium Collectio...,Diana Ross,2000-01-01,217493,46,0.401,0.384000,0.783,0.003190,0.4680,-5.929,0.0465,99.274,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100750,7jPTUOj9L1dNcsF8Rn3LOy,Idea,Tomorrow,The O'My's,2018-09-07,236571,34,0.609,0.053900,0.619,0.001460,0.0888,-7.189,0.1880,168.255,4.0
100751,6AJlcxjEO2baFC24GPsJjg,"Holding Out for a Hero - From ""Footloose"" Soun...",Ravishing - The Best Of,Bonnie Tyler,2009-03-30,348706,1,0.575,0.015900,0.811,0.000004,0.0902,-10.981,0.0401,149.671,4.0
100752,0Wif6GC74X6oWrGrPoru4K,Daddy Cool,The Essential Boney M.,Boney M.,2012-08-27,207893,35,0.764,0.270000,0.775,0.767000,0.1560,-6.907,0.0368,124.424,4.0
100753,1YGKUHiQ6b4I6yXy9IONc5,Everybody Needs Somebody To Love - Mono / Rema...,The Rolling Stones In Mono (Remastered 2016),The Rolling Stones,1966-01-01,304706,0,0.533,0.236000,0.781,0.022500,0.1540,-5.665,0.0740,177.917,4.0
